In [373]:
import pandas as pd

In [493]:
df = pd.read_csv('data/csv/dk.csv', parse_dates=['collected_at'])

In [495]:
df['americanOdds'] = (
    df['americanOdds']
    .astype(str)
    .str.replace('−', '-', regex=False)
    .str.replace(r'^\+', '', regex=True)
    .astype(int)
)

In [497]:
def american_to_prob(o):
    if o > 0:
        return 100 / (o + 100)
    else:
        return -o / (abs(o) + 100)
df['imp_prob'] = df['americanOdds'].apply(american_to_prob)

In [499]:
df.head()

,id,marketId,label,americanOdds,decimalOdds,fractionalOdds,trueOdds,outcomeType,sortOrder,tags,main,collected_at,imp_prob
0,0HC79840267N150_3,2_79840267,CLE Guardians,136,2.36,34/25,2.360000,Away,-1501,MainPointLine;SGP,True,2025-05-06 14:47:36.831000+00:00,0.423729
1,0HC79832764N150_3,2_79832764,LA Dodgers,-170,1.58,10/17,1.588235,Away,-1501,MainPointLine;SGP,True,2025-05-06 14:47:36.839000+00:00,0.629630
2,0HC79832765N150_3,2_79832765,CLE Guardians,136,2.36,34/25,2.360000,Away,-1501,MainPointLine;SGP,True,2025-05-06 14:47:36.839000+00:00,0.423729
3,0HC79832766N150_3,2_79832766,TEX Rangers,136,2.36,34/25,2.360000,Away,-1501,MainPointLine;SGP,True,2025-05-06 14:47:36.839000+00:00,0.423729
4,0HC79832807N150_3,2_79832807,PHI Phillies,140,2.40,7/5,2.400000,Away,-1501,MainPointLine;SGP,True,2025-05-06 14:47:36.839000+00:00,0.416667


In [501]:
df['sum_prob'] = df.groupby(
    ['marketId', 'collected_at']
)['imp_prob'].transform('sum')

df['fair_prob'] = df['imp_prob'] / df['sum_prob']

In [503]:
df.head()

,id,marketId,label,americanOdds,decimalOdds,fractionalOdds,trueOdds,outcomeType,sortOrder,tags,main,collected_at,imp_prob,sum_prob,fair_prob
0,0HC79840267N150_3,2_79840267,CLE Guardians,136,2.36,34/25,2.360000,Away,-1501,MainPointLine;SGP,True,2025-05-06 14:47:36.831000+00:00,0.423729,0.423729,1.000000
1,0HC79832764N150_3,2_79832764,LA Dodgers,-170,1.58,10/17,1.588235,Away,-1501,MainPointLine;SGP,True,2025-05-06 14:47:36.839000+00:00,0.629630,1.042853,0.603757
2,0HC79832765N150_3,2_79832765,CLE Guardians,136,2.36,34/25,2.360000,Away,-1501,MainPointLine;SGP,True,2025-05-06 14:47:36.839000+00:00,0.423729,1.042049,0.406630
3,0HC79832766N150_3,2_79832766,TEX Rangers,136,2.36,34/25,2.360000,Away,-1501,MainPointLine;SGP,True,2025-05-06 14:47:36.839000+00:00,0.423729,1.042049,0.406630
4,0HC79832807N150_3,2_79832807,PHI Phillies,140,2.40,7/5,2.400000,Away,-1501,MainPointLine;SGP,True,2025-05-06 14:47:36.839000+00:00,0.416667,1.040727,0.400361


In [505]:
df['collected_at'] = pd.to_datetime(df['collected_at']).dt.floor('s')

In [541]:
dk_ts = df.pivot_table(
    index='collected_at',
    columns=['marketId', 'label'],
    values='fair_prob'
).sort_index()

In [543]:
clean = dk_ts.dropna(how='all')
clean.columns = [
    f"{label} @ {marketId}"
    for marketId, label in clean.columns
]
clean = clean.round(3)
clean = clean.reset_index() 

In [545]:
dk_ts = clean
dk_ts

,collected_at,LA Dodgers @ 1_79832764,MIA Marlins @ 1_79832764,CLE Guardians @ 1_79832765,WAS Nationals @ 1_79832765,BOS Red Sox @ 1_79832766,TEX Rangers @ 1_79832766,PHI Phillies @ 1_79832807,TB Rays @ 1_79832807,NY Yankees @ 1_79832816,...,Over @ 3_79832893,Under @ 3_79832893,Over @ 3_79832894,Under @ 3_79832894,Over @ 3_79832913,Under @ 3_79832913,Over @ 3_79832914,Under @ 3_79832914,Over @ 3_79840267,Under @ 3_79840267
0,2025-05-06 14:47:36+00:00,0.7,0.3,0.517,0.483,0.478,0.522,0.526,0.474,0.496,...,0.496,0.504,0.504,0.496,0.511,0.489,0.496,0.504,0.483,0.517


In [525]:
df_kalshi = pd.read_csv('kalshi_sports_prices.csv', parse_dates=['timestamp'])

In [527]:
df_kalshi['ks_prob'] = df_kalshi['last_price'] / 100.0
df_kalshi = df_kalshi.rename(columns={'timestamp': 'collected_at', 'market_ticker':'marketId'})
df_kalshi['collected_at'] = (
    pd.to_datetime(df_kalshi['collected_at']).dt.floor('s')
)

In [529]:
df_kalshi

,collected_at,event_ticker,yes_sub_title,marketId,yes_bid,yes_ask,no_bid,no_ask,last_price,ks_prob
0,2025-05-06 18:42:14,KXMLB-25,Arizona,KXMLB-25-ARI,2,3,97,98,3,0.03
1,2025-05-06 18:42:14,KXMLB-25,Atlanta,KXMLB-25-ATL,6,7,93,94,7,0.07
2,2025-05-06 18:42:14,KXMLB-25,Baltimore,KXMLB-25-BAL,1,2,98,99,2,0.02
3,2025-05-06 18:42:14,KXMLB-25,Boston,KXMLB-25-BOS,4,5,95,96,5,0.05
4,2025-05-06 18:42:14,KXMLB-25,Chicago C,KXMLB-25-CHC,5,6,94,95,5,0.05
...,...,...,...,...,...,...,...,...,...,...
117,2025-05-06 18:42:28,KXMLBGAME-25MAY06LADMIA,Los Angeles D,KXMLBGAME-25MAY06LADMIA-LAD,70,72,28,30,72,0.72
118,2025-05-06 18:42:28,KXMLBGAME-25MAY06CLEWSHG1,Washington,KXMLBGAME-25MAY06CLEWSHG1-WSH,47,48,52,53,49,0.49
119,2025-05-06 18:42:28,KXMLBGAME-25MAY06CLEWSHG1,Cleveland,KXMLBGAME-25MAY06CLEWSHG1-CLE,51,54,46,49,54,0.54
120,2025-05-06 18:42:28,KXMLBGAME-25MAY05CLEWSH,Washington,KXMLBGAME-25MAY05CLEWSH-WSH,46,49,51,54,49,0.49


In [423]:
ks_ts = df_kalshi.pivot_table(
    index='collected_at',
    columns='marketId',
    values='ks_prob',
    aggfunc='first'
).sort_index()

In [425]:
ks_ts

marketId,KXMLBGAME-25MAY05CLEWSH-CLE,KXMLBGAME-25MAY05CLEWSH-WSH,KXMLBGAME-25MAY06BALMIN-BAL,KXMLBGAME-25MAY06BALMIN-MIN,KXMLBGAME-25MAY06CINATL-ATL,KXMLBGAME-25MAY06CINATL-CIN,KXMLBGAME-25MAY06CLEWSHG1-CLE,KXMLBGAME-25MAY06CLEWSHG1-WSH,KXMLBGAME-25MAY06CWSKC-CWS,KXMLBGAME-25MAY06CWSKC-KC,...,KXMLBGAME-25MAY06SDNYY-NYY,KXMLBGAME-25MAY06SDNYY-SD,KXMLBGAME-25MAY06SEAATH-ATH,KXMLBGAME-25MAY06SEAATH-SEA,KXMLBGAME-25MAY06SFCHC-CHC,KXMLBGAME-25MAY06SFCHC-SF,KXMLBGAME-25MAY06TEXBOS-BOS,KXMLBGAME-25MAY06TEXBOS-TEX,KXMLBGAME-25MAY06TORLAA-LAA,KXMLBGAME-25MAY06TORLAA-TOR
collected_at,,,,,,,,,,,,,,,,,,,,,
2025-05-06 16:23:02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.53,0.49,NaN,NaN,NaN,NaN,0.49,NaN
2025-05-06 16:23:03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.62,0.39,NaN,NaN,NaN,0.51
2025-05-06 16:23:04,NaN,NaN,0.39,0.61,0.67,NaN,NaN,NaN,0.34,0.67,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-05-06 16:23:05,NaN,NaN,NaN,NaN,NaN,0.34,NaN,NaN,NaN,NaN,...,0.52,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-05-06 16:23:06,0.55,0.48,NaN,NaN,NaN,NaN,0.52,0.5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.49,0.53,NaN,NaN


In [547]:
dk_long = (
    dk_ts
    .reset_index()                     
    .melt(
        id_vars='collected_at',
        var_name='market_col',        
        value_name='dk_prob'
    )
)

In [551]:
dk_long['yes_sub_title'] = (
    dk_long['market_col']
    .str.split(r'\s*@\s*')            # split on " @ "
    .str[0]                           # take the left part: "LA Dodgers", etc.
    .str.strip()
)

In [553]:
dk_long

,collected_at,market_col,dk_prob,yes_sub_title
0,2025-05-06 14:47:36+00:00,index,0.000,index
1,2025-05-06 14:47:36+00:00,LA Dodgers @ 1_79832764,0.700,LA Dodgers
2,2025-05-06 14:47:36+00:00,MIA Marlins @ 1_79832764,0.300,MIA Marlins
3,2025-05-06 14:47:36+00:00,CLE Guardians @ 1_79832765,0.517,CLE Guardians
4,2025-05-06 14:47:36+00:00,WAS Nationals @ 1_79832765,0.483,WAS Nationals
...,...,...,...,...
92,2025-05-06 14:47:36+00:00,Under @ 3_79832913,0.489,Under
93,2025-05-06 14:47:36+00:00,Over @ 3_79832914,0.496,Over
94,2025-05-06 14:47:36+00:00,Under @ 3_79832914,0.504,Under
95,2025-05-06 14:47:36+00:00,Over @ 3_79840267,0.483,Over


In [597]:
# All unique subtitles in DK data
dk_titles = dk_long['yes_sub_title'].unique()
print("DK subtitles:", dk_titles)

# All unique subtitles Kalshi data
ks_titles = df_kalshi['yes_sub_title'].unique()
print("Kalshi subtitles:", ks_titles)

DK subtitles: ['index' 'LA Dodgers' 'MIA Marlins' 'CLE Guardians' 'WAS Nationals'
 'BOS Red Sox' 'TEX Rangers' 'PHI Phillies' 'TB Rays' 'NY Yankees'
 'SD Padres' 'ATL Braves' 'CIN Reds' 'CHI Cubs' 'SF Giants' 'BAL Orioles'
 'MIN Twins' 'CHI White Sox' 'KC Royals' 'HOU Astros' 'MIL Brewers'
 'PIT Pirates' 'STL Cardinals' 'COL Rockies' 'DET Tigers' 'LA Angels'
 'TOR Blue Jays' 'ARI Diamondbacks' 'NY Mets' 'Athletics' 'SEA Mariners'
 'Over' 'Under']
Kalshi subtitles: ['Arizona' 'Atlanta' 'Baltimore' 'Boston' 'Chicago C' 'Chicago WS'
 'Cincinnati' 'Cleveland' 'Colorado' 'Detroit' 'Miami' 'Houston'
 'Kansas City' 'Los Angeles A' 'Los Angeles D' 'Milwaukee' 'Minnesota'
 'New York M' 'New York Y' "A's" 'Philadelphia' 'Pittsburgh' 'San Diego'
 'San Francisco' 'Seattle' 'St. Louis' 'Tampa Bay' 'Texas' 'Toronto'
 'Washington' 'Chicago W']


In [599]:
dk_to_ks = {
    # MLB teams
    'LA Dodgers'        : 'Los Angeles D',
    'MIA Marlins'       : 'Miami',
    'CLE Guardians'     : 'Cleveland',
    'WAS Nationals'     : 'Washington',
    'BOS Red Sox'       : 'Boston',
    'TEX Rangers'       : 'Texas',
    'PHI Phillies'      : 'Philadelphia',
    'TB Rays'           : 'Tampa Bay',
    'NY Yankees'        : 'New York Y',
    'SD Padres'         : 'San Diego',
    'ATL Braves'        : 'Atlanta',
    'CIN Reds'          : 'Cincinnati',
    'CHI Cubs'          : 'Chicago C',
    'CHI White Sox'     : 'Chicago WS',
    'SF Giants'         : 'San Francisco',
    'BAL Orioles'       : 'Baltimore',
    'MIN Twins'         : 'Minnesota',
    'KC Royals'         : 'Kansas City',
    'HOU Astros'        : 'Houston',
    'MIL Brewers'       : 'Milwaukee',
    'PIT Pirates'       : 'Pittsburgh',
    'STL Cardinals'     : 'St. Louis',
    'COL Rockies'       : 'Colorado',
    'DET Tigers'        : 'Detroit',
    'LA Angels'         : 'Los Angeles A',
    'TOR Blue Jays'     : 'Toronto',
    'ARI Diamondbacks'  : 'Arizona',
    'NY Mets'           : 'New York M',
    "Athletics"         : "A's",       # Oakland A’s in Kalshi
    # anything you don’t care to join—map to None or leave out
    'index'             : None,
    'Over'              : None,
    'Under'             : None,
}

In [601]:
dk_long['yes_sub_title_norm'] = dk_long['yes_sub_title'].map(dk_to_ks)

In [603]:
dk_for_merge = dk_long.dropna(subset=['yes_sub_title_norm'])

In [605]:
merged = pd.merge(
    dk_for_merge,
    df_kalshi,
    left_on='yes_sub_title_norm',
    right_on='yes_sub_title',
    how='inner'
)

In [607]:
merged

,collected_at_x,market_col,dk_prob,yes_sub_title_x,yes_sub_title_norm,collected_at_y,event_ticker,yes_sub_title_y,marketId,yes_bid,yes_ask,no_bid,no_ask,last_price,ks_prob
0,2025-05-06 14:47:36+00:00,LA Dodgers @ 1_79832764,0.7,LA Dodgers,Los Angeles D,2025-05-06 18:42:15,KXMLB-25,Los Angeles D,KXMLB-25-LAD,26,27,73,74,27,0.27
1,2025-05-06 14:47:36+00:00,LA Dodgers @ 1_79832764,0.7,LA Dodgers,Los Angeles D,2025-05-06 18:42:18,KXMLBNL-25,Los Angeles D,KXMLBNL-25-LAD,37,39,61,63,37,0.37
2,2025-05-06 14:47:36+00:00,LA Dodgers @ 1_79832764,0.7,LA Dodgers,Los Angeles D,2025-05-06 18:42:20,KXMLBNLWEST-25,Los Angeles D,KXMLBNLWEST-25-LAD,78,81,19,22,79,0.79
3,2025-05-06 14:47:36+00:00,LA Dodgers @ 1_79832764,0.7,LA Dodgers,Los Angeles D,2025-05-06 18:42:28,KXMLBGAME-25MAY06LADMIA,Los Angeles D,KXMLBGAME-25MAY06LADMIA-LAD,70,72,28,30,72,0.72
4,2025-05-06 14:47:36+00:00,MIA Marlins @ 1_79832764,0.3,MIA Marlins,Miami,2025-05-06 18:42:15,KXMLB-25,Miami,KXMLB-25-FLA,0,1,99,100,1,0.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,2025-05-06 14:47:36+00:00,WAS Nationals @ 2_79840267,1.0,WAS Nationals,Washington,2025-05-06 18:42:17,KXMLB-25,Washington,KXMLB-25-WAS,0,1,99,100,1,0.01
250,2025-05-06 14:47:36+00:00,WAS Nationals @ 2_79840267,1.0,WAS Nationals,Washington,2025-05-06 18:42:18,KXMLBNL-25,Washington,KXMLBNL-25-WAS,0,1,99,100,1,0.01
251,2025-05-06 14:47:36+00:00,WAS Nationals @ 2_79840267,1.0,WAS Nationals,Washington,2025-05-06 18:42:21,KXMLBNLEAST-25,Washington,KXMLBNLEAST-25-WSH,0,5,95,100,2,0.02
252,2025-05-06 14:47:36+00:00,WAS Nationals @ 2_79840267,1.0,WAS Nationals,Washington,2025-05-06 18:42:28,KXMLBGAME-25MAY06CLEWSHG1,Washington,KXMLBGAME-25MAY06CLEWSHG1-WSH,47,48,52,53,49,0.49


In [613]:
clean = (
    merged
    [['collected_at_x','yes_sub_title_norm','dk_prob','ks_prob']]
    .rename(columns={
        'collected_at':'time_collected',
        'yes_sub_title_norm':'team',
        'dk_prob':'dk_prob',
        'ks_prob':'ks_prob'
    })
)
clean.head(10)

,collected_at_x,team,dk_prob,ks_prob
0,2025-05-06 14:47:36+00:00,Los Angeles D,0.700,0.27
1,2025-05-06 14:47:36+00:00,Los Angeles D,0.700,0.37
2,2025-05-06 14:47:36+00:00,Los Angeles D,0.700,0.79
3,2025-05-06 14:47:36+00:00,Los Angeles D,0.700,0.72
4,2025-05-06 14:47:36+00:00,Miami,0.300,0.01
5,2025-05-06 14:47:36+00:00,Miami,0.300,0.01
6,2025-05-06 14:47:36+00:00,Miami,0.300,0.01
7,2025-05-06 14:47:36+00:00,Miami,0.300,0.29
8,2025-05-06 14:47:36+00:00,Cleveland,0.517,0.02
9,2025-05-06 14:47:36+00:00,Cleveland,0.517,0.04
